In [224]:
from tika import parser
import re
import pandas as pd
import numpy as np
from googletrans import Translator
from glob import glob
from sklearn.cluster import KMeans

In [180]:
translator = Translator()

In [181]:
def clean_text(offer,raw,df=pd.DataFrame(columns=["offer","string"])):
    print(offer)
    pattern = r"[^\w]"
    l = np.array([])
    raw = translator.translate(raw["content"]).text
    for i in raw.split("."):
        if (i!=''):
            l = np.append(l,(re.sub(' +',' ',re.sub(pattern," ",i))))
    return df.append(pd.DataFrame({"offer":offer,"string":l}))

In [182]:
df=pd.DataFrame(columns=["offer","string"])
for i in glob("./Offres d'emploi_Inputs/PDF/*"):
    raw = parser.from_file(i)
    df=clean_text(i.split("\\")[1],raw,df)

Offre1.pdf
Offre10.pdf
Offre11.pdf
Offre12.pdf
Offre13.pdf
Offre14.pdf
Offre15.pdf
Offre16.pdf
Offre17.pdf
Offre18.pdf
Offre19.pdf
Offre2.pdf
Offre20.pdf
Offre21.pdf
Offre22.pdf
Offre3.pdf
Offre4.pdf
Offre5.pdf
Offre6.pdf
Offre7.pdf
Offre8.pdf
Offre9.pdf


In [183]:
df

,offer,string
0,Offre1.pdf,JURISTE IT H F Full description General inform...
1,Offre1.pdf,On 1 January 2019 more than 1500 employees wo...
2,Offre1.pdf,CISA Crédit Agricole Assurances CACIB Crédit ...
3,Offre1.pdf,Our ambitions to develop new platforms adapte...
4,Offre1.pdf,Crédit Agricole Group Infrastructure Platform...
5,Offre1.pdf,Job location Geographical area Europe France ...
6,Offre1.pdf,Required skills Behavioral Competencies good ...
7,Offre1.pdf,IT tools Control of office tools
8,Offre1.pdf,languages Functional English
0,Offre10.pdf,STAGE Legal PPI M F Rueil CONTRATStage MÉTIERJ...


In [184]:
embeddings_index = {}
embeddings_vectors = []
f = open('./glove.6B.100d.txt', 'rb')

word_idx = 0

for line in f:
    values = line.decode('utf-8').split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = word_idx
    embeddings_vectors.append(vector)
    word_idx = word_idx + 1
f.close()

inv_index = {v: k for k, v in embeddings_index.items()}
print(word_idx)

400000


In [185]:
glove_embeddings = np.vstack(embeddings_vectors)
glove_norms = np.linalg.norm(glove_embeddings, axis=-1, keepdims=True)
glove_embeddings_normed = glove_embeddings / glove_norms
print(glove_embeddings.shape)

(400000, 100)


In [186]:
def get_emb(word):
    idx = embeddings_index.get(word)
    if idx is None:
        return 0
    else:
        return glove_embeddings[idx]

In [187]:
def sentence_emb(words, topn=10):
    query_emb = 0
    for word in words:
        query_emb += get_emb(word)       
        
    query_emb = query_emb / np.linalg.norm(query_emb)
    return query_emb

In [210]:
df_dim = pd.DataFrame()
for i in df.string:
    df_dim = df_dim.append(pd.DataFrame(data=sentence_emb(i.split()).reshape(1,-1)))

C:\Users\Justpro\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [219]:
df_dim = df_dim.reset_index(drop=True)

In [220]:
df = df.reset_index(drop=True)

In [222]:
final = pd.concat([df,df_dim],axis=1)

In [225]:
final.head()

,offer,string,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,Offre1.pdf,JURISTE IT H F Full description General inform...,-0.059559,-0.019263,0.038895,0.052929,0.055720,-0.128761,0.029971,0.086423,...,0.047246,-0.083189,0.012098,0.075108,0.031596,0.045481,-0.043336,-0.118600,0.209990,0.028483
1,Offre1.pdf,On 1 January 2019 more than 1500 employees wo...,0.005201,0.053058,0.031084,0.004862,0.010609,0.004671,-0.025991,0.073778,...,0.038785,-0.001593,-0.011827,0.031500,-0.153755,0.034962,-0.036745,-0.039736,0.169013,0.015525
2,Offre1.pdf,CISA Crédit Agricole Assurances CACIB Crédit ...,-0.002383,0.005118,0.083725,-0.019112,0.057146,-0.004849,-0.071836,-0.023300,...,-0.011920,0.006874,-0.067320,0.067908,-0.132959,-0.028581,-0.051282,-0.091482,0.167524,0.054506
3,Offre1.pdf,Our ambitions to develop new platforms adapte...,-0.002316,-0.005631,0.063622,0.009073,0.052814,-0.024759,-0.087944,-0.005644,...,-0.023104,-0.003725,-0.050268,-0.004527,-0.101034,0.010293,-0.059626,-0.101406,0.171112,0.053154
4,Offre1.pdf,Crédit Agricole Group Infrastructure Platform...,-0.009818,0.029202,0.053980,-0.027293,0.039541,-0.022820,-0.063090,0.027805,...,-0.016041,-0.010274,-0.047476,0.002294,-0.083120,0.028751,-0.042972,-0.070934,0.171406,0.046760


In [242]:
kmeans = KMeans(n_clusters=8, random_state=0)

In [231]:
final = final.dropna().reset_index(drop=True)

In [243]:
kmeans.fit(final.iloc[:,2:])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [244]:
final["cluster"] = kmeans.labels_

In [248]:
final[final["cluster"] == 3]

,offer,string,0,1,2,3,4,5,6,7,...,91,92,93,94,95,96,97,98,99,cluster
6,Offre1.pdf,Required skills Behavioral Competencies good ...,0.027085,0.051712,0.013204,0.063209,-0.025841,-0.030241,-0.029901,-0.072901,...,-0.038545,-0.009416,0.117475,-0.009748,0.037387,-0.102393,-0.230332,0.141456,0.178010,3
8,Offre1.pdf,languages Functional English,0.001844,0.155152,0.017389,0.102634,0.122094,0.085581,-0.020920,-0.136534,...,0.027093,-0.006397,0.081630,-0.122983,0.038315,-0.233700,-0.126970,0.002570,-0.005830,3
29,Offre11.pdf,You will carry out studies give advice and wi...,-0.042728,0.069808,0.026825,-0.041584,0.002523,0.021961,-0.030951,0.043294,...,-0.046953,-0.007602,-0.072883,-0.093196,0.033953,-0.066674,-0.108959,0.150847,0.043041,3
30,Offre11.pdf,You will also likely bring assistance to netw...,-0.019010,0.027021,0.043490,-0.011019,0.048060,-0.066340,-0.046972,0.037697,...,-0.009073,-0.056937,-0.053842,-0.101466,0.018791,0.015349,-0.021444,0.165732,0.044339,3
31,Offre11.pdf,All our positions are open to people with dis...,-0.026811,0.087848,0.055245,-0.014400,-0.013501,0.063953,-0.099164,0.052560,...,-0.013575,-0.003162,0.003969,-0.134744,0.025776,-0.071236,-0.077621,0.178993,0.057748,3
33,Offre11.pdf,Required skills Anaytical and synthetical min...,0.022645,0.000902,0.078120,-0.035595,-0.022225,-0.002416,-0.025350,-0.017199,...,-0.041089,-0.035177,0.094725,-0.097334,0.034527,-0.090040,-0.107725,0.143290,0.129832,3
34,Offre11.pdf,IT tools Good computer skills pack office leg...,-0.040464,0.020983,0.025172,-0.023138,0.055957,-0.046922,-0.008030,-0.058221,...,0.033271,0.095602,0.013888,0.033668,-0.036193,-0.013998,-0.115789,0.237356,0.074400,3
41,Offre12.pdf,Within a dedicated pole you incorporate a wea...,-0.042059,0.057331,0.023128,-0.060798,0.046648,0.014920,-0.028239,0.004765,...,-0.066722,-0.002906,0.002262,-0.070008,0.012067,-0.065849,-0.105372,0.161818,0.066211,3
45,Offre12.pdf,This experience in banking will allow you to ...,-0.000888,0.055300,0.042779,0.007085,-0.021359,-0.044712,-0.086561,0.011153,...,0.027231,0.017091,-0.030218,-0.034945,0.050615,-0.023036,-0.123181,0.161882,0.094128,3
46,Offre12.pdf,A successful alternation may lead to great op...,-0.023777,0.012550,0.020029,0.017274,0.040671,-0.036943,-0.011745,-0.034050,...,-0.032405,-0.038407,0.018032,-0.098351,-0.001411,-0.064305,-0.087460,0.087310,0.022524,3
